Random Forest
 
Assignment


About the data: 
Let’s consider a Company dataset with around 10 variables and 400 records. 
The attributes are as follows: 
 Sales -- Unit sales (in thousands) at each location
 Competitor Price -- Price charged by competitor at each location
 Income -- Community income level (in thousands of dollars)
 Advertising -- Local advertising budget for company at each location (in thousands of dollars)
 Population -- Population size in region (in thousands)
 Price -- Price company charges for car seats at each site
 Shelf Location at stores -- A factor with levels Bad, Good and Medium indicating the quality of the shelving location for the car seats at each site
 Age -- Average age of the local population
 Education -- Education level at each location
 Urban -- A factor with levels No and Yes to indicate whether the store is in an urban or rural location
 US -- A factor with levels No and Yes to indicate whether the store is in the US or not
The company dataset looks like this: 
 
Problem Statement:
A cloth manufacturing company is interested to know about the segment or attributes causes high sale. 
Approach - A Random Forest can be built with target variable Sales (we will first convert it in categorical variable) & all other variable will be independent in the analysis.  


In [24]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import KFold,cross_val_score
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [2]:
data_df = pd.read_csv('Company_Data.csv')
data_df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [3]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 34.5+ KB


In [4]:
report = pp.ProfileReport(data_df)
report.to_widgets()

In [10]:
#As described in problem statment we will first convert target variable Sale to categorical variable.As targetVar is continuos and we 
#need to convert to categorical so we use cut func where it will calculate the mean of that col and construct bins as defined
n_bins=2
label_encoder = LabelEncoder()
data_df['Sales_Label'] = label_encoder.fit_transform(pd.cut(data_df.Sales, n_bins, retbins=True)[0])

# data_df = pd.get_dummies(data_df) or
data_df['ShelveLoc_label'] = label_encoder.fit_transform(data_df['ShelveLoc'])
data_df['Urban_label'] = label_encoder.fit_transform(data_df['Urban'])
data_df['US_label'] = label_encoder.fit_transform(data_df['US'])
data_df.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US,Sales_Label,ShelveLoc_label,Urban_label,US_label
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes,1,0,1,1
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes,1,1,1,1
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes,1,2,1,1
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes,0,2,1,1
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No,0,0,1,0


In [15]:
#Feature Selection Model Validation
x = data_df.iloc[:,[1,2,3,4,5,7,8,12,13,14]]
y=data_df.iloc[:,11]
model =DecisionTreeClassifier(criterion='entropy')# as the dependent var is categorical so we can use either c5.0(entropy) or CART(gini impurities)
model.fit(x,y)
rfe = RFE(model)
rfe.fit(x,y)
print("Decession Tree Feature selection \n",model.feature_importances_)
print("Recursive Feature Selection Ranking ", rfe.ranking_,"\n Recursive Feature Selection Top Features ",rfe.support_)

Decession Tree Feature selection 
 [0.17965296 0.09066376 0.09007251 0.02188692 0.27783818 0.13664536
 0.         0.15553983 0.01377891 0.03392155]
Recursive Feature Selection Ranking  [1 1 1 4 1 2 6 1 5 3] 
 Recursive Feature Selection Top Features  [ True  True  True False  True False False  True False False]


In [49]:
#From the above feature selection method we can select top 5 features for getting the best fit model(which are dependent to predict y vriable)
featured_x = x.iloc[:,[0,1,2,4,5,7]]
std_xValues = StandardScaler().fit_transform(featured_x.values)
num_trees = 300
max_features = 4
kfold = KFold(n_splits=10, random_state=7)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, std_xValues, y, cv=kfold)
print(results.mean())

C:\Users\Akshay\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


0.8099999999999999


In [50]:

print("Mean---",results.mean(),"\n Std Deviation----",results.std())
y_pred = model.fit(std_xValues,y).predict(featured_x)
print(classification_report(y,y_pred))
cnf_matrix = confusion_matrix(y,y_pred)
print(cnf_matrix)

Mean--- 0.8099999999999999 
 Std Deviation---- 0.047696960070847276
              precision    recall  f1-score   support

           0       0.70      0.44      0.54       241
           1       0.46      0.72      0.56       159

    accuracy                           0.55       400
   macro avg       0.58      0.58      0.55       400
weighted avg       0.61      0.55      0.55       400

[[107 134]
 [ 45 114]]


In [51]:

passedTest = data_df.iloc[np.where(y_pred==y)]
print("Total Test length is ",len(featured_x),"\n Passed Test count is ",len(passedTest))
failed = data_df.iloc[np.where(y_pred!=y)]
print("\n"," Failed Test count is",len(failed))

Total Test length is  400 
 Passed Test count is  221

  Failed Test count is 179
